In [7]:
import os
import asyncio
import yaml
from dotenv import load_dotenv

# Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.search_engine import GoogleConnector
from semantic_kernel.core_plugins import WebSearchEnginePlugin
from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

# Updated imports for chat history and message content
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

In [8]:
load_dotenv()

True

## Creating the Kernel

In [9]:

# Here we are adding  AzureChatCompletion service and a Google WebSearch plugin as a tool.
def create_kernel(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    
    # Get and configure the prompt execution settings for auto function calling.
    settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
    
    # Add the Google WebSearch plugin (if API keys are set)
    google_api_key = os.getenv("GOOGLE_API_KEY")
    google_search_engine_id = "44b41c05ad10f4559"  # TODO fget this from env variables
    if google_api_key and google_search_engine_id:
        connector = GoogleConnector(
            api_key=google_api_key,
            search_engine_id=google_search_engine_id
        )
        web_plugin = WebSearchEnginePlugin(connector)
        kernel.add_plugin(web_plugin, plugin_name="WebSearch")
    return kernel


# Loading the Instructions for the Agents fom the config files

In [10]:

# Load YAML configurations for agents and tasks
def load_configurations():
    with open("../crewai/config/agents.yaml", "r") as f:
        agents_config = yaml.safe_load(f)
    with open("../crewai/config/tasks.yaml", "r") as f:
        tasks_config = yaml.safe_load(f)
    return agents_config, tasks_config

# Build full instructions for an agent from its config and the corresponding task.
def build_agent_instructions(agent_config: dict, task_config: dict) -> str:
    # Concatenate role, goal, and backstory
    instructions = (
        f"Role: {agent_config.get('role','')}\n"
        f"Goal: {agent_config.get('goal','')}\n"
        f"Backstory: {agent_config.get('backstory','')}\n\n"
        f"Task:\n{task_config.get('description','')}\n\n"
        f"Expected Output:\n{task_config.get('expected_output','')}\n"
    )
    return instructions


In [11]:
# Helper to collect responses from an async generator by creating a ChatHistory
async def collect_response(agent: ChatCompletionAgent, input_text: str) -> str:
    # Create a new ChatHistory object with the agent's instructions as system message.
    history = ChatHistory(system_message=agent.instructions)
    # Add a user message with the provided input text.
    history.add_message(ChatMessageContent(role=AuthorRole.USER, content=input_text))
    
    response_text = ""
    async for result in agent.invoke(history):
        response_text += result.content
    return response_text


## Create the Agents

In [16]:

#TODO handle the visual content generation
async def run_market_news_monitor(subject: str, agent_instr: str, kernel: Kernel) -> str:
    # Create the Market News Monitor agent
    agent = ChatCompletionAgent(
        service_id="market_news",  # service id for this agent
        kernel=kernel,
        name="MarketNewsMonitor",
        instructions=agent_instr,
        arguments=KernelArguments()  # use default settings here
    )
    # Pass the subject as input.
    input_text = f"Subject: {subject}"
    response = await collect_response(agent, input_text)
    print("=== Market News Monitor Output ===")
    print(response)
    return response

async def run_data_analyst(subject: str, agent_instr: str, kernel: Kernel) -> str:
    # Create the Data Analyst agent
    agent = ChatCompletionAgent(
        service_id="data_analyst",
        kernel=kernel,
        name="DataAnalyst",
        instructions=agent_instr,
        arguments=KernelArguments()
    )
    input_text = f"Subject: {subject}"
    response = await collect_response(agent, input_text)
    print("=== Data Analyst Output ===")
    print(response)
    return response

async def run_content_creator(subject: str, combined_context: dict, agent_instr: str, kernel: Kernel) -> str:
    # Create the Content Creator agent
    agent = ChatCompletionAgent(
        service_id="content_creator",
        kernel=kernel,
        name="ContentCreator",
        instructions=agent_instr,
        arguments=KernelArguments()
    )
    # Build input that includes both the subject and the outputs from previous agents.
    input_text = (
        f"Subject: {subject}\n\n"
        f"Market News Analysis:\n{combined_context['market_news']}\n\n"
        f"Data Analysis:\n{combined_context['data_analyst']}\n\n"
        "Based on the above, create high-quality content."
    )
    response = await collect_response(agent, input_text)
    print("=== Content Creator Output ===")
    print(response)
    return response

async def run_quality_assurance(content: str, agent_instr: str, kernel: Kernel) -> str:
    # Create the Quality Assurance agent (Content Officer)
    agent = ChatCompletionAgent(
        service_id="quality_assurance",
        kernel=kernel,
        name="ContentOfficer",
        instructions=agent_instr,
        arguments=KernelArguments()
    )
    input_text = (
        "Please review and refine the following content ensuring it is formatted in markdown "
        "and structured as JSON with two keys: 'article' and 'social_media_posts'. "
        "The 'social_media_posts' value should be a list of objects, each with 'platform' and 'content'.\n\n"
        f"Content:\n{content}"
    )
    response = await collect_response(agent, input_text)
    print("=== Quality Assurance Output ===")
    print(response)
    return response


## Defining the Flow

In [13]:
async def flow(subject: str):
    # Load YAML configurations
    agents_config, tasks_config = load_configurations()

    # Subject for the content creation
    subject = subject

    # Build instructions for each agent by combining agent and task configs.
    market_news_instr = build_agent_instructions(
        agents_config["market_news_monitor_agent"],
        tasks_config["monitor_financial_news"]
    )
    data_analyst_instr = build_agent_instructions(
        agents_config["data_analyst_agent"],
        tasks_config["analyze_market_data"]
    )
    content_creator_instr = build_agent_instructions(
        agents_config["content_creator_agent"],
        tasks_config["create_content"]
    )
    quality_assurance_instr = build_agent_instructions(
        agents_config["quality_assurance_agent"],
        tasks_config["quality_assurance"]
    )

    # TODO see if single kernel works here instead creating seprate kernels
    kernel_market = create_kernel("market_news")
    kernel_data = create_kernel("data_analyst")
    kernel_content = create_kernel("content_creator")
    kernel_quality = create_kernel("quality_assurance")

    # Run the first two agents in parallel (Market News Monitor and Data Analyst)
    market_news_future = run_market_news_monitor(subject, market_news_instr, kernel_market)
    data_analyst_future = run_data_analyst(subject, data_analyst_instr, kernel_data)
    market_news_result, data_analyst_result = await asyncio.gather(market_news_future, data_analyst_future)

    # Combine the outputs from the first two agents
    combined_context = {
        "market_news": market_news_result,
        "data_analyst": data_analyst_result
    }

    # Run Content Creator agent with the combined context
    content_output = await run_content_creator(subject, combined_context, content_creator_instr, kernel_content)

    # Run Quality Assurance agent on the content output
    final_output = await run_quality_assurance(content_output, quality_assurance_instr, kernel_quality)

    # Print final structured output (should be a JSON-like string with 'article' and 'social_media_posts')
    print("\n=== FINAL STRUCTURED OUTPUT ===")
    print(final_output)
    return final_output


## Kiking off the flow

In [14]:

final_output = await flow("Inflation in the US and the impact on the stock market in 2024")

=== Market News Monitor Output ===
### Summary Report on Inflation in the US and its Impact on the Stock Market in 2024

#### 1. Current Inflation Trends in the US
- **Recent CPI Data**: The latest Consumer Price Index (CPI) data shows a moderate increase of 2.8% year-over-year for October 2023, slightly higher than September’s 2.6%. This indicates a gradual upward trend in inflation.
- **Core Inflation**: Excluding food and energy, core inflation remains stable at 2.5%, suggesting that the rise in inflation is more deeply rooted in the broader economy rather than temporary shocks.

#### 2. Economic Indicators and Their Implications
- **Federal Reserve's Stance**: Recent statements from the Federal Reserve suggest a cautious approach, with potential rate hikes being considered in early 2024 if inflation doesn't stabilize. This could tighten financial conditions and impact cost of capital for businesses.
- **Wage Growth**: Moderate wage growth at around 3% continues, but it is not outpa

## show the content in formatted way

In [15]:

import json, re, textwrap
from IPython.display import display, Markdown

def extract_json(text: str) -> str:
    """
    Extract JSON content from a string that may be wrapped in markdown code fences.
    """
    pattern = re.compile(r"```json\s*(\{.*\})\s*```", re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(1)
    return text.strip()

def render_article_to_markdown(article: dict) -> str:
    """
    Converts the article dictionary (with keys like 'title', 'introduction', 'sections')
    into a Markdown-formatted string.
    """
    md = ""
    title = article.get("title", "")
    if title:
         md += f"# {title}\n\n"
    intro = article.get("introduction", "")
    if intro:
         md += f"{intro}\n\n"
    sections = article.get("sections", [])
    for section in sections:
         header = section.get("header", "")
         content = section.get("content", "")
         bullets = section.get("bullets", [])
         if header:
             md += f"## {header}\n\n"
         if content:
             md += f"{content}\n\n"
         if bullets:
             for bullet in bullets:
                  md += f"- {bullet}\n"
             md += "\n"
    return md

if final_output is None:
    print("Final output is None. Please check that your QA agent is producing an output.")
else:
    try:
        # Clean the output by extracting JSON if it's wrapped in code fences
        final_output_cleaned = extract_json(final_output)
        output_data = json.loads(final_output_cleaned)
        
        article = output_data.get("article", "")
        posts = output_data.get("social_media_posts", [])
        
        # If the article is a dict, convert it to Markdown text.
        if isinstance(article, dict):
            article_md = render_article_to_markdown(article)
        else:
            article_md = article
        
        # Display the blog post as Markdown
        display(Markdown(article_md))
        
        # Print the social media posts
        for post in posts:
            platform = post.get("platform", "")
            content = post.get("content", "")
            print(platform)
            wrapped_content = textwrap.fill(content, width=50)
            print(wrapped_content)
            print('-' * 50)
    except Exception as e:
        print("Error parsing final output:", e)


# Navigating Inflation and the Stock Market in 2024: Strategies for Smart Investing

In the intricate ebb and flow of economic cycles, inflation presents both challenges and opportunities for investors. As 2024 approaches, understanding the dynamics of inflation in the United States becomes pivotal, especially in light of its ripple effects on the stock market. Based on the latest insights from our market news analysis and data specialists, this article delves into the heart of inflation as an economic issue, empowering investors with the knowledge and strategies to navigate the turbulent waters of inflation.

## Current Inflation Landscape

Inflation in the United States has shown a moderate increase, with October 2023's Consumer Price Index (CPI) reporting a 2.8% year-over-year rise. This increment slightly surpasses September’s 2.6%, indicating a gradual but persistent trend. Notably, core inflation, which excludes volatile food and energy sectors, remains stable at 2.5%. These figures suggest that inflation is deeply rooted in broader economic conditions rather than being driven by temporary disruptions.

**Visual Aid:**
- Embed a [line graph illustrating the trajectory of CPI from 2020 to October 2023](#), providing a visual representation of inflation trends.

## Economic Indicators: A Cautious Outlook

### Federal Reserve's Position

The Federal Reserve's recent statements reflect a cautious approach, contemplating potential interest rate hikes in early 2024 if inflation does not ease. As interest rates rise, the cost of capital for businesses follows suit, impacting their financial strategies and growth prospects. This careful balancing act by the Fed remains crucial to maintaining economic stability.

### Wage Growth and Consumer Behavior

With moderate wage growth hovering around 3%, consumer spending patterns remain steady yet cautious. This balanced but prudent spending affects sectors reliant on discretionary income, such as retail and automotive. Understanding these subtle shifts in consumer behavior will be key for investors seeking to align with resilient sectors.

## Stock Market Reactions and Strategic Shifts

Increased market volatility is evident as investors weigh potential interest rate hikes. Growth-heavy sectors, particularly technology, feel the pinch under increased rates. This has initiated a notable sector rotation, directing investments towards defensive areas like utilities and consumer staples, with growing interest in dividend-paying stocks as a hedge against inflation.

**Visual Aid:**
- Include an [infographic showcasing sector performance under varying inflation scenarios](#). Highlight which sectors traditionally outperform during times of rising inflation.

## Preparing for 2024: Actionable Strategies

1. **Embrace Defensive Sectors:** Analyze performance trends and focus on defensive investment strategies in utilities, consumer staples, and dividend-paying stocks. These sectors can offer stability amidst inflationary pressures.

2. **Stay Informed on Fed Policies:** Keep abreast of updates on Federal Reserve meetings and policy changes. Understanding the Fed's moves can provide clarity on potential market reactions and inform investment strategies effectively.

3. **Global Insight Through Comparisons:** Comparing the U.S. inflation scenario with those of other major economies provides global context. Overseas variations in inflation rates can influence global investment flows and present unique opportunities for strategic investments.

**Visual Aid:**
- A [world map infographic depicting inflation rates across key global markets](#), illustrating comparative perspectives.

## Risks and Opportunities

### Risks

With the possibility of further rate hikes, there’s a looming risk of a “hard-landing” scenario, where aggressive monetary tightening could stymie growth and spike market instability. Investors must remain vigilant to avoid potential pitfalls in their investment strategies.

### Opportunities

Shifting investments towards sectors with pricing power and strong fundamentals can yield attractive returns. Additionally, market volatility may present lucrative merger and acquisition opportunities within affected sectors, offering strategic investors additional entry points.

## Conclusion

Understanding the nuances of inflation and its impact on the stock market is crucial as 2024 unfolds. By aligning investment strategies with market conditions, investors can better navigate risks and seize opportunities. Staying informed on economic indicators, sector trends, and Fed policies will be pivotal in making informed, strategic decisions.

**Call to Action:**
- Encourage readers to [subscribe for further updates and insights](#), ensuring they remain equipped to face the economic uncertainties of 2024.


Twitter
🔍 Trend Alert: US inflation inches upward as
October CPI shows a 2.8% rise. Understand how this
impacts your investment strategy for 2024 with our
latest insights. #InflationWatch #InvestSmart2024
--------------------------------------------------
LinkedIn
Inflation is on the move, and its impact on the
stock market can't be overlooked. As sectors
adjust, learn how you can navigate these shifts
with strategic insights. Discover which sectors to
focus on for robust portfolio growth in our new
blog post. #InvestmentStrategy #StockMarket2024
--------------------------------------------------
Instagram
[Image of stock market graph] Navigating the
inflation curve? 📈 Our latest analysis sheds light
on how inflation affects stock market performance
and what smart investors can do in 2024. Tap the
link in bio to read more! #FinanceTips
#EconomicInsights
--------------------------------------------------
